In [1]:
import os

import numpy as np
import pandas as pd

from scipy import stats

import rioxarray

import xarray as xr
import geopandas as gpd

In [2]:
wb_path = "../../Bases_de_datos/CCKP_NetCDF/"
map_path = "../../Bases_de_datos/Mapas/WB_countries_Admin0_10m"
#map_path = "../../Bases_de_datos/Mapas/Natural_Earth/ne_10m_admin_0_countries"
wb_files = os.listdir(wb_path)
for i, f in enumerate(wb_files):
    if f[0] in ["~", "."]: wb_files.remove(f)
for i, f in enumerate(wb_files): print(f"{i}: {f}")

gdf = gpd.read_file(map_path)
gdf["Area"] = gdf.to_crs(8857).area
gdf.head()#.plot()

0: climatology-popcount-annual-mean_pop-x0.25_gpw-v4-rev11-ssp245_climatology_mean_2040-2059.nc
1: climatology-spei12-annual-mean_cmip6-x0.25_ensemble-all-historical_climatology_median_1995-2014.nc
2: climatology-popcount-annual-mean_pop-x0.25_gpw-v4-rev11-historical_climatology_mean_1995-2014.nc
3: climatology-spei12-annual-mean_cmip6-x0.25_ensemble-all-ssp245_climatology_median_2040-2059.nc


,OBJECTID,featurecla,LEVEL,TYPE,FORMAL_EN,FORMAL_FR,POP_EST,POP_RANK,GDP_MD_EST,POP_YEAR,...,NAME_TR,NAME_VI,NAME_ZH,WB_NAME,WB_RULES,WB_REGION,Shape_Leng,Shape_Area,geometry,Area
0,1,Admin-0 country,2,Sovereign country,Republic of Indonesia,None,260580739,17,3028000.0,2017,...,Endonezya,Indonesia,印度尼西亚,Indonesia,None,EAP,495.029918,153.078608,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4...",1.879821e+12
1,2,Admin-0 country,2,Sovereign country,Malaysia,None,31381992,15,863000.0,2017,...,Malezya,Malaysia,马来西亚,Malaysia,None,EAP,68.456913,26.703172,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4...",3.278847e+11
2,3,Admin-0 country,2,Sovereign country,Republic of Chile,None,17789267,14,436100.0,2017,...,Şili,Chile,智利,Chile,None,LCR,416.997272,76.761813,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611...",7.365975e+11
3,4,Admin-0 country,2,Sovereign country,Plurinational State of Bolivia,None,11138234,14,78350.0,2017,...,Bolivya,Bolivia,玻利維亞,Bolivia,None,LCR,54.345991,92.203587,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5...",1.086808e+12
4,5,Admin-0 country,2,Sovereign country,Republic of Peru,None,31036656,15,410400.0,2017,...,Peru,Peru,秘鲁,Peru,None,LCR,73.262192,106.417089,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832...",1.289867e+12


In [3]:
p = "climatology-popcount-annual-mean"

i = 2
pop_h = xr.open_dataset(wb_path + wb_files[i])
pop_h = pop_h.rio.write_crs(gdf.crs)
i = 0
pop_f = xr.open_dataset(wb_path + wb_files[i])
pop_f = pop_f.rio.write_crs(gdf.crs)

pop_f

<xarray.Dataset> Size: 4MB
Dimensions:                           (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                              (time) datetime64[ns] 8B 2040-07-01
  * lat                               (lat) float64 6kB -90.0 -89.75 ... 90.0
  * lon                               (lon) float64 12kB -180.0 -179.8 ... 179.8
  * bnds                              (bnds) int32 8B 0 1
    spatial_ref                       int64 8B 0
Data variables:
    climatology-popcount-annual-mean  (time, lat, lon) float32 4MB ...
    lat_bnds                          (lat, bnds) float64 12kB ...
    lon_bnds                          (lon, bnds) float64 23kB ...
Attributes: (12/20)
    wb_truncation_label:   2
    wb_grid_label:         x0.25
    wb_period_label:       2040-2059
    wb_percentile_label:   mean
    wb_type_label:         climatology
    wb_model_label:        gpw-v4
    ...                    ...
    Conventions:           CF-1.0
    Source_Software:       Esri ArcGIS
    creation_date:         Tue Nov 21 05:53:44 PM MST 2023
    user:                  file standardized by Caspar Ammann, Climate Strate...
    data_source:           Gridded Population of the World, Version 4: GPWv4;...
    title:                 Gridded Population of the World, Version 4, Revisi...

In [4]:
gdf_b = gdf[gdf["Area"] >= 1e9]
pop_h["country"] = pop_h[p].copy()

for i in gdf_b.index:
    ds_c = xr.full_like( pop_h[[p]],
        int(gdf_b.loc[gdf_b.index == i, "ISO_N3"].iloc[0]) )
    pop_w = ds_c.rio.clip( gdf_b[gdf_b.index == i].geometry,
        drop = False, invert = False,all_touched = False )
    pop_h["country"] = pop_h["country"].where( pop_w[p].isnull(), pop_w[p] )

for l in range(2):
    iso = pop_h["country"].values
    iso_c  = iso.copy()
    for i in range(iso.shape[1]):
        for j in range(iso.shape[2]):

            i_l = i - 1
            i_p = i + 1
            j_l = j - 1
            j_p = j + 1

            if i_l < 0: i_l = iso.shape[1] - 1
            if i_p == iso.shape[1]: i_p = 0
            if j_l < 0: j_l = iso.shape[2] - 1
            if j_p == iso.shape[2]: j_p = 0

            adj = [0] * 8
            adj[0] = iso[0, i_l, j_p]
            adj[1] = iso[0, i, j_p]
            adj[2] = iso[0, i_p, j_p]
            adj[3] = iso[0, i_l, j]
            cell =  iso[0, i, j]
            adj[4] = iso[0, i_p, j]
            adj[5] = iso[0, i_l, j_l]
            adj[6] = iso[0, i, j_l]
            adj[7] = iso[0, i_p, j_l]

            if np.isnan(cell) or (cell in gdf["ISO_N3"].astype(int).values):
                pass
            else:
                adj_c = []
                for k in adj:
                    if k in gdf["ISO_N3"].astype(int).values: adj_c.append(k)
                    if len(adj_c) > 0:
                        m = stats.mode(adj_c)[0]
                        if not (type(m) == np.float32): m = m[0]
                        iso_c[0, i, j] = int(m)

    pop_h["country"] = ( ("time", "lat", "lon"), iso_c )

gdf_s = gdf[gdf["Area"] < 1e9]
a = 0

for i in gdf_s.index: 
    ds_c = xr.full_like( pop_h[["climatology-popcount-annual-mean"]],
        int(gdf_s.loc[gdf_s.index == i, "ISO_N3"].iloc[0]) )
    pop_w = ds_c.rio.clip( gdf_s[gdf_s.index == i].geometry,
        drop = False, invert = False, all_touched = True )
    pop_h["country"] = pop_h["country"].where( pop_w[p].isnull(), pop_w[p] )

iso = pop_h["country"].values
iso_c  = iso.copy()

for i in range(iso.shape[1]):
    for j in range(iso.shape[2]):
        cell = iso_c[0, i, j]
        if cell in gdf["ISO_N3"].astype(int).values: pass
        else: iso_c[0, i, j] = np.nan

pop_h["country"] = ( ("time", "lat", "lon"), iso_c )

#pop_h.to_netcdf("a.nc")
df_pop_h = pop_h[ [p, "country"] ].to_dataframe()

pop_h

<xarray.Dataset> Size: 8MB
Dimensions:                           (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                              (time) datetime64[ns] 8B 1995-07-01
  * lat                               (lat) float64 6kB -90.0 -89.75 ... 90.0
  * lon                               (lon) float64 12kB -180.0 -179.8 ... 179.8
  * bnds                              (bnds) int32 8B 0 1
    spatial_ref                       int64 8B 0
Data variables:
    climatology-popcount-annual-mean  (time, lat, lon) float32 4MB nan ... nan
    lat_bnds                          (lat, bnds) float64 12kB ...
    lon_bnds                          (lon, bnds) float64 23kB ...
    country                           (time, lat, lon) float32 4MB nan ... nan
Attributes: (12/20)
    wb_truncation_label:   2
    wb_grid_label:         x0.25
    wb_period_label:       1995-2014
    wb_percentile_label:   mean
    wb_type_label:         climatology
    wb_model_label:        gpw-v4
    ...                    ...
    Conventions:           CF-1.0
    Source_Software:       Esri ArcGIS
    creation_date:         Tue Nov 21 03:44:21 PM MST 2023
    user:                  file standardized by Caspar Ammann, Climate Strate...
    data_source:           Gridded Population of the World, Version 4: GPWv4;...
    title:                 Gridded Population of the World, Version 4, Revisi...

In [5]:
gdf_b = gdf[gdf["Area"] >= 1e9]
pop_f["country"] = pop_f[p].copy()

for i in gdf_b.index:
    ds_c = xr.full_like( pop_f[[p]],
        int(gdf_b.loc[gdf_b.index == i, "ISO_N3"].iloc[0]) )
    pop_w = ds_c.rio.clip( gdf_b[gdf_b.index == i].geometry,
        drop = False, invert = False,all_touched = False )
    pop_f["country"] = pop_f["country"].where( pop_w[p].isnull(), pop_w[p] )

for l in range(2):
    iso = pop_f["country"].values
    iso_c  = iso.copy()
    for i in range(iso.shape[1]):
        for j in range(iso.shape[2]):

            i_l = i - 1
            i_p = i + 1
            j_l = j - 1
            j_p = j + 1

            if i_l < 0: i_l = iso.shape[1] - 1
            if i_p == iso.shape[1]: i_p = 0
            if j_l < 0: j_l = iso.shape[2] - 1
            if j_p == iso.shape[2]: j_p = 0

            adj = [0] * 8
            adj[0] = iso[0, i_l, j_p]
            adj[1] = iso[0, i, j_p]
            adj[2] = iso[0, i_p, j_p]
            adj[3] = iso[0, i_l, j]
            cell =  iso[0, i, j]
            adj[4] = iso[0, i_p, j]
            adj[5] = iso[0, i_l, j_l]
            adj[6] = iso[0, i, j_l]
            adj[7] = iso[0, i_p, j_l]

            if np.isnan(cell) or (cell in gdf["ISO_N3"].astype(int).values):
                pass
            else:
                adj_c = []
                for k in adj:
                    if k in gdf["ISO_N3"].astype(int).values: adj_c.append(k)
                    if len(adj_c) > 0:
                        m = stats.mode(adj_c)[0]
                        if not (type(m) == np.float32): m = m[0]
                        iso_c[0, i, j] = int(m)

    pop_f["country"] = ( ("time", "lat", "lon"), iso_c )

gdf_s = gdf[gdf["Area"] < 1e9]
a = 0

for i in gdf_s.index: 
    ds_c = xr.full_like( pop_f[["climatology-popcount-annual-mean"]],
        int(gdf_s.loc[gdf_s.index == i, "ISO_N3"].iloc[0]) )
    pop_w = ds_c.rio.clip( gdf_s[gdf_s.index == i].geometry,
        drop = False, invert = False, all_touched = True )
    pop_f["country"] = pop_f["country"].where( pop_w[p].isnull(), pop_w[p] )

iso = pop_f["country"].values
iso_c  = iso.copy()

for i in range(iso.shape[1]):
    for j in range(iso.shape[2]):
        cell = iso_c[0, i, j]
        if cell in gdf["ISO_N3"].astype(int).values: pass
        else: iso_c[0, i, j] = np.nan

pop_f["country"] = ( ("time", "lat", "lon"), iso_c )

#pop_f.to_netcdf("a.nc")
df_pop_f = pop_f[ [p, "country"] ].to_dataframe()

pop_f

<xarray.Dataset> Size: 8MB
Dimensions:                           (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                              (time) datetime64[ns] 8B 2040-07-01
  * lat                               (lat) float64 6kB -90.0 -89.75 ... 90.0
  * lon                               (lon) float64 12kB -180.0 -179.8 ... 179.8
  * bnds                              (bnds) int32 8B 0 1
    spatial_ref                       int64 8B 0
Data variables:
    climatology-popcount-annual-mean  (time, lat, lon) float32 4MB nan ... nan
    lat_bnds                          (lat, bnds) float64 12kB ...
    lon_bnds                          (lon, bnds) float64 23kB ...
    country                           (time, lat, lon) float32 4MB nan ... nan
Attributes: (12/20)
    wb_truncation_label:   2
    wb_grid_label:         x0.25
    wb_period_label:       2040-2059
    wb_percentile_label:   mean
    wb_type_label:         climatology
    wb_model_label:        gpw-v4
    ...                    ...
    Conventions:           CF-1.0
    Source_Software:       Esri ArcGIS
    creation_date:         Tue Nov 21 05:53:44 PM MST 2023
    user:                  file standardized by Caspar Ammann, Climate Strate...
    data_source:           Gridded Population of the World, Version 4: GPWv4;...
    title:                 Gridded Population of the World, Version 4, Revisi...

In [20]:
v = "climatology-spei12-annual-mean"

i = 1
ds_h = xr.open_dataset(wb_path + wb_files[i])
ds_h[v] = ds_h[v].where( ds_h[v] <= -0.25, 0 )
ds_h[v] = ds_h[v].where( ds_h[v] == 0, 1 )
ds_h["Exposed population"] = pop_h[p] * ds_h[v]

i = 3
ds_f = xr.open_dataset(wb_path + wb_files[i])
ds_f[v] = ds_f[v].where( ds_f[v] <= -0.25, 0 )
ds_f[v] = ds_f[v].where( ds_f[v] == 0, 1 )
ds_f["Exposed population"] = pop_f[p] * ds_f[v]

ds_h

<xarray.Dataset> Size: 8MB
Dimensions:                         (time: 1, lat: 721, lon: 1440, bnds: 2)
Coordinates:
  * time                            (time) datetime64[ns] 8B 1995-07-01
  * lat                             (lat) float64 6kB -90.0 -89.75 ... 90.0
  * lon                             (lon) float64 12kB -180.0 -179.8 ... 179.8
  * bnds                            (bnds) int32 8B 0 1
    spatial_ref                     int64 8B 0
Data variables:
    climatology-spei12-annual-mean  (time, lat, lon) float32 4MB 0.0 0.0 ... 0.0
    lon_bnds                        (lon, bnds) float64 23kB ...
    lat_bnds                        (lat, bnds) float64 12kB ...
    Exposed population              (time, lat, lon) float32 4MB nan nan ... nan
Attributes: (12/43)
    wb:                     \nwb_data_citation = Please acknowledge data sour...
    climate_data_id:       CMIP6-x0.25
    ensemble_citation:     Please acknowledge ensemble data source as \nWorld...
    creation_date:         2023-05-15T01:00:00Z
    ensemble_author:       Caspar M Ammann, Climate Strategies
    experiment:            CMIP6: CMIP all-forcing simulations
    ...                    ...
    wb_model_label:        ensemble
    wb_type_label:         climatology
    wb_percentile_label:   median
    wb_period_label:       1995-2014
    wb_grid_label:         x0.25
    wb_truncation_label:   2

In [21]:
df_h = ds_h[ [v, "Exposed population"] ].to_dataframe()
df_h["country"] = df_pop_h["country"]
df_h[p] = df_pop_h[p]
df_h = df_h.reset_index().drop(columns = "time").set_index("country")
df_h = df_h[ ~df_h.index.isnull() ]
df_hc = df_h[ [p, "Exposed population"] ].groupby("country").sum()
df_hc["% Exposed population"] = ( df_hc["Exposed population"] / df_hc[p] )
df_hc.sort_values("% Exposed population", ascending = False )

,climatology-popcount-annual-mean,Exposed population,% Exposed population
country,,,
64.0,1.579239e+06,1.579239e+06,1.000000
520.0,1.110906e+04,1.110906e+04,1.000000
666.0,6.421992e+03,6.421992e+03,1.000000
728.0,7.351284e+06,7.297417e+06,0.992672
344.0,7.174090e+06,7.120242e+06,0.992494
...,...,...,...
86.0,0.000000e+00,0.000000e+00,NaN
239.0,0.000000e+00,0.000000e+00,NaN
260.0,0.000000e+00,0.000000e+00,NaN


In [19]:
df_f = ds_f[ [v, "Exposed population"] ].to_dataframe()
df_f["country"] = df_pop_f["country"]
df_f[p] = df_pop_f[p]
df_f = df_f.reset_index().drop(columns = "time").set_index("country")
df_f = df_f[ ~df_f.index.isnull() ]
df_fc = df_f[ [p, "Exposed population"] ].groupby("country").sum()
df_fc["% Exposed population"] = ( df_fc["Exposed population"] / df_fc[p] )
df_fc.sort_values("% Exposed population", ascending = False)

,climatology-popcount-annual-mean,Exposed population,% Exposed population
country,,,
36.0,35920644.0,3.108134e+06,0.086528
710.0,63008264.0,4.654370e+06,0.073869
12.0,48973440.0,1.441670e+06,0.029438
152.0,20354390.0,2.157373e+05,0.010599
76.0,230181184.0,8.772661e+05,0.003811
...,...,...,...
86.0,0.0,0.000000e+00,NaN
239.0,0.0,0.000000e+00,NaN
260.0,0.0,0.000000e+00,NaN


In [11]:
vn = "% Population exposed to an average annual SPEI < -0.25, change 2000-2060"
df_fc[vn] = df_fc["% Exposed population"] - df_hc["% Exposed population"]
names = gdf[ ["WB_NAME", "WB_A3", "ISO_N3"] ].drop_duplicates(
    subset = "ISO_N3" ).set_index("ISO_N3")
names.index = names.index.astype(int)
df_fc["WB_NAME"] = names["WB_NAME"]
df_fc["WB_A3"] = names["WB_A3"]

df_fc.sort_values(vn, ascending = False)

,climatology-popcount-annual-mean,Exposed population,% Exposed population,"% Population exposed to an average annual SPEI < -0.25, change 2000-2060",WB_NAME,WB_A3
country,,,,,,
12.0,48973440.0,1.458475e+07,0.297809,0.297809,Algeria,DZA
36.0,35920644.0,4.712560e+06,0.131194,0.131194,Australia,AUS
710.0,63008264.0,5.242920e+06,0.083210,0.083210,South Africa,ZAF
894.0,29440262.0,2.210510e+06,0.075085,0.075085,Zambia,ZMB
516.0,3360020.0,2.227059e+05,0.066281,0.066281,Namibia,NAM
...,...,...,...,...,...,...
86.0,0.0,0.000000e+00,NaN,NaN,British Indian Ocean Territory (UK),IOT
239.0,0.0,0.000000e+00,NaN,NaN,South Georgia and South Sandwich Islands (UK),SGS
260.0,0.0,0.000000e+00,NaN,NaN,French Southern and Antarctic Lands (Fr.),ATF
